# An Autoencoder's Compression to Measure the Complexity of Text

I developed this idea while working on a project that measures the quality of EU legislation. The core concept of this notebook is to exploit the compression quality of an autoencoder as an indication of the complexity of a sentence. This idea aims to contribute to the NLP literature by measuring complexity in text.

I propose to use an autoencoder model that consists of two parts: an encoder and decoder. The encoder encodes an input into a lower embedding space than the original input size through several non-linear hidden layers. This is the concept of a bottleneck, which has the lowest layer size. From this bottleneck, the decoder model has to reproduce the original input with just the information available at the bottleneck. The compression takes place exactly at this bottleneck. The replicated sentence is then compared to the original input, ensuring that the input equals the output. A simple autoencoder architecture is illustrated below (from Google Images).

The idea, to reiterate, is that more complex sentences are harder to compress, which is associated with a higher loss when passing this sentence during inference time through the model. I will first load a sample sentence from a Wikipedia dataset, then build and train the model, and finally illustrate with mock sentences that the above idea indeed works.

I plan to extend the method by adding further enhancements to the model (dropout layers, layer normalization, learning rate decay, etc.) to optimize the autoencoder.

In [8]:
# Load packages
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
import numpy as np 
import random

# For
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# For efficien computing
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


## Data

Here I used a Wikipedia dataset. I will use the EU legislation set next to see the methods application to our data.

In [11]:
# Load data 
with open('/Users/gerritquaremba/Library/CloudStorage/GoogleDrive-g.quaremba@gmail.com/My Drive/08_NN_Kaparthy/data/wikisent2.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [12]:
# Split the data into a list of setnences
text = text.split('\n')

# Shuffle and select x sentences
random.seed(42)
random.shuffle(text)
text=text[:1000]

# DO: shuffle at this stage 

# Keep only sentences of a certain size and lower each
text = [t.lower() for t in text if len(t)<=400]
text[:3]

['imanol sarriegi isasa (born 27 april 1995) is a spanish footballer who plays as a central midfielder.',
 'indeed, the greater part of this chisian daniel cannot be said to deserve the name of a translation at all.',
 'after moving to los angeles to become an actor, rambo started working in the real estate business.']

In [13]:
# Number of sentences for training
len(text)

1000

Currently, we are on char-level, can also change this of course.

In [14]:
# Set of unique chars
chars = set(''.join(text))

# Length of chars
vocab_size = len(chars)
vocab_size

56

In [15]:
# Int to chat
int2char = dict(enumerate(chars))

# Char to int
char2int = {char: ind for ind, char in int2char.items()}

In [16]:
# Longest sentence
maxlen = len(max(text, key=len))
maxlen

384

In [17]:
# Apply padding
for i in range(len(text)):
  while len(text[i])<maxlen:
      text[i] += ' '

In [18]:
# Check sentences
text[:2]

['imanol sarriegi isasa (born 27 april 1995) is a spanish footballer who plays as a central midfielder.                                                                                                                                                                                                                                                                                           ',
 'indeed, the greater part of this chisian daniel cannot be said to deserve the name of a translation at all.                                                                                                                                                                                                                                                                                     ']

Define the input and output tensors, which are the same for the autoencoder!

In [19]:
# Get in and out
inout = []

for i in range(len(text)):
    inout.append([char2int[character] for character in text[i]])

In [20]:
# Create a tensor
inout = torch.tensor(inout, dtype=torch.long)
inout.shape

torch.Size([1000, 384])

`inout` is of size `(B, T)`. We have set `T` above to be a bit shorter. A batch is an epoch here.

## Autoencoder Architecture

Below implements the AC. The optimal architecture is to be determined. Currently, the bottleneck is of dim `(8,8)`.

In [23]:
# AE
class AE(nn.Module): # inherent from the nn class

    def __init__(self):
        super().__init__()
        
        # Create embedding matrix
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed) # This is a 65 x n_embed matrix; weights are inintialised at random
        
    
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(n_embed, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, n_embed)
        )
   
    def forward(self, idx):

       # Get embeddings
        embeddings = self.token_embedding_table(idx)

        # Encode
        encoded = self.encoder(embeddings)

        # Decode
        reconstructed = self.decoder(encoded)

        # Compute loss
        loss = F.mse_loss(reconstructed, embeddings)

        return reconstructed, loss

In [24]:
# Initialise model and HPs
# Hyperparameters
n_embed=64
batch_size=len(text)
max_iters = 500
learning_rate=1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
model = AE()
m = model.to(device)

In [25]:
# Training

# PyTorch optimiser
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for step in range(max_iters):

    # Forward pass
    _, loss = model(inout)

    # Set gradients to 0
    optimizer.zero_grad(set_to_none=True) # more efficient

    # Backprop
    loss.backward()

    # Adam, update gradients
    optimizer.step()

    # Print final loss (batch loss)
    if step % 100  == 0:
        print(f"Step[{step}/{max_iters}] loss: {loss.item()}")

Step[0/500] loss: 1.1515729427337646
Step[100/500] loss: 0.18843428790569305
Step[200/500] loss: 0.09664537757635117
Step[300/500] loss: 0.050881076604127884
Step[400/500] loss: 0.02575305663049221


From observing the training loss we cann tell that the optimisation works. But there is considerable room for improvement!

### Compression as a measure for text complexity?

The intuition is that a sentence that is more complex (longer, more complex syntax, etc) is more difficult to compress and and rebuild, whicch is indicated with a higher loss during inference. Below I test exaclty this. For the evaluation, I took a simple comparsion of the loss. But a more intuitive measure would be to take the logits and compute the probability of the sentence. (Is this entropy?)

In [29]:
# Method to get loss for a sentence
def compression_score(model, sent):


    model.eval() # eval mode
  
    # Pre sentence for input into the mdoel
    sent = sent.lower()

    # Apply padding
    while len(sent)<maxlen:
      sent += ' '

    # Sentecne to idx, and tensor with batch dim 1
    sent_idx = [char2int[character] for character in sent] # 
    sent_idx = torch.tensor(sent_idx, dtype=torch.long).view(1, maxlen)
    
    # Output the logits
    _, loss = model(sent_idx)
    
    return loss.item()

In [30]:
# Comparison of scores
compression_score(model, "Hello there") < compression_score(model, "Hello there this is likely more difficult, becasue it contains more text.")

True

The losses differ considerably:

In [31]:
# Difficult sentence
compression_score(model, "Hello there. This is likely more difficult, because it contains more text.")

0.009428826160728931

In [32]:
# Easier sentence
compression_score(model, "Hello there")

0.0009569653775542974

One concern that may drive the above result is that sheer length is driving the comeplexity of a sentence. Let us test same length but more complicated syntax/grammar.

In [33]:
# Same length, different complexity
compression_score(model, "This is a sentence that has the same length as the other sentence.") < compression_score(model, "This is, by far, a more convoluted sentence; hence more complex.")

True

In [34]:
# Easy sentence
compression_score(model, "This is a sentence that has the same length as the other sentence.")

0.004809746518731117

In [35]:
# Complex sentence
compression_score(model, "This is, by far, a more convoluted sentence; hence more complex.")

0.008957922458648682

Seems to look good! More in the near future!